In [2]:
import Pkg
Pkg.add("Clustering")

   Resolving package versions...
   Installed NearestNeighbors ─ v0.4.16
   Installed Clustering ─────── v0.15.7
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [aaaa29a8] + Clustering v0.15.7
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [aaaa29a8] + Clustering v0.15.7
  [b8a86587] + NearestNeighbors v0.4.16
Precompiling project...
  ✓ NearestNeighbors
  ✓ Clustering
  2 dependencies successfully precompiled in 24 seconds. 395 already precompiled.


In [4]:
using Clustering
using CSV
using DataFrames
using Statistics
using StatsBase
using TimeSeries

In [91]:
acwi = Matrix(CSV.read("../sb_gbi/equity_scenarios.csv",DataFrame))

99999×50 Matrix{Float64}:
  0.183608   0.040727   0.130638  …   0.310636   0.259417   0.106576
  0.057796  -0.015669   0.01473       0.181475   0.038817   0.09112
 -0.042034   0.071062   0.211914      0.271774   0.03051    0.014144
  0.048233   0.002585   0.069304      0.161997   0.150916  -0.458586
 -0.097277   0.278737   0.249458      0.054621   0.20249    0.148626
  0.168582   0.058298   0.174671  …  -0.057467  -0.037438   0.030723
  0.096997   0.214256  -0.246285      0.148997   0.192833   0.181174
  0.254872   0.054849   0.014113      0.13077    0.066608  -0.076274
  0.263792   0.213822   0.260164      0.309639   0.022386   0.268776
  0.065249  -0.206639   0.347267      0.165357   0.180862   0.298468
  ⋮                               ⋱                        
  0.304079  -0.004624   0.041851  …  -0.008278   0.136057   0.139732
  0.261117   0.094383  -0.240846      0.037133  -0.072158  -0.037526
 -0.050816   0.211829  -0.100711      0.158417  -0.021944   0.318735
  0.370563   0.279

In [92]:
inflation = Matrix(CSV.read("../sb_gbi/inflation_scenarios.csv", DataFrame))[:,2:end]

99999×50 Matrix{Float64}:
 0.035158  0.031416  0.046096  0.018379  …  0.06628   0.09403   0.093851
 0.013222  0.0123    0.043899  0.027086     0.019199  0.021789  0.020151
 0.039912  0.040263  0.035394  0.0123       0.02551   0.0123    0.0123
 0.016048  0.0123    0.0123    0.03366      0.056582  0.0123    0.0123
 0.065174  0.052422  0.071214  0.05068      0.019615  0.058837  0.048566
 0.051986  0.035766  0.026192  0.033541  …  0.052188  0.046143  0.026802
 0.040925  0.025968  0.043467  0.024486     0.0786    0.047548  0.027347
 0.0123    0.045685  0.022954  0.0123       0.019471  0.048676  0.043244
 0.0123    0.014093  0.0123    0.0123       0.037708  0.071717  0.083529
 0.0123    0.046767  0.048754  0.071964     0.058765  0.0123    0.024597
 ⋮                                       ⋱                      
 0.05639   0.034089  0.038381  0.037493  …  0.037054  0.031733  0.02699
 0.022314  0.042175  0.031607  0.039552     0.060712  0.045391  0.080075
 0.078099  0.090769  0.052886  0.02624

In [93]:
returns = stack([acwi,inflation],dims=1)

2×99999×50 Array{Float64, 3}:
[:, :, 1] =
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

[:, :, 2] =
 0.040727  -0.015669  0.071062  0.002585  …  0.095772  -0.393472  0.014187
 0.031416   0.0123    0.040263  0.0123       0.058533   0.059451  0.027505

[:, :, 3] =
 0.130638  0.01473   0.211914  0.069304  …  -0.292217  0.060243  0.174296
 0.046096  0.043899  0.035394  0.0123        0.036324  0.051567  0.0123

;;; … 

[:, :, 48] =
 0.310636  0.181475  0.271774  0.161997  …  0.084165  0.142273  0.012147
 0.06628   0.019199  0.02551   0.056582     0.0123    0.04997   0.065018

[:, :, 49] =
 0.259417  0.038817  0.03051  0.150916  …  0.118251  0.023003  0.20869
 0.09403   0.021789  0.0123   0.0123       0.035531  0.0123    0.095034

[:, :, 50] =
 0.106576  0.09112   0.014144  -0.458586  …  0.292593  0.115302  0.170179
 0.093851  0.020151  0.0123     0.0123       0.045336  0.0123    0.057752

In [94]:
period_1 = returns[:,:,1] 


2×99999 Matrix{Float64}:
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

In [95]:
period_1[:,1]

2-element Vector{Float64}:
 0.183608
 0.035158

In [96]:
n_clusters = 12
result = kmeans(period_1,n_clusters)

KmeansResult{Matrix{Float64}, Float64, Int64}([0.2253154523059055 -0.14079174898456562 … -0.05810467428222364 0.17308855652609606; 0.04085042841738455 0.04103953432169 … 0.04050950201588278 0.04004276358559344], [12, 6, 11, 6, 11, 12, 8, 1, 9, 6  …  7, 9, 9, 11, 4, 2, 2, 10, 2, 7], [0.00013451960628781123, 0.0006286982387657229, 0.0002586235805443083, 0.0006610547443635104, 0.002142808562948087, 0.00016294994777329985, 0.0007880190231008173, 0.0016887164744094768, 0.0012449934299754162, 0.0006739110406751465  …  0.0006443437671831474, 0.0006265395534228169, 0.0008883691151426454, 0.0014660951314903001, 0.0001964695172221309, 0.00038241597153208795, 0.00038339450721935897, 2.3065076592720946e-5, 0.00021384525345737038, 0.001827495161968983], [12065, 4913, 1434, 4900, 2040, 13727, 11452, 10800, 8710, 7808, 8178, 13972], [12056, 4924, 1434, 4902, 2046, 13734, 11459, 10775, 8708, 7809, 8185, 13967], 81.89174815574594, 100, false)

In [97]:
counts(result)

12-element Vector{Int64}:
 12065
  4913
  1434
  4900
  2040
 13727
 11452
 10800
  8710
  7808
  8178
 13972

In [98]:
centers =result.centers

2×12 Matrix{Float64}:
 0.225315   -0.140792   0.46779    …  0.114375   -0.0581047  0.173089
 0.0408504   0.0410395  0.0404449     0.0655032   0.0405095  0.0400428

In [99]:
for cluster in 1:n_clusters
    println("equity: $(centers[1,cluster]), infl: $(centers[2,cluster])")
end

equity: 0.2253154523059055, infl: 0.04085042841738455
equity: -0.14079174898456562, infl: 0.04103953432169
equity: 0.46778991143654186, infl: 0.04044485634588581
equity: 0.3578912794777632, infl: 0.03986341187270549
equity: -0.2552018929618774, infl: 0.040349679374389165
equity: 0.06161205191495527, infl: 0.03800376722003677
equity: 0.006428221223492391, infl: 0.040918832533379045
equity: 0.12001683972157773, infl: 0.024859322227379157
equity: 0.2843649441892517, infl: 0.040966136770785234
equity: 0.1143753703419134, infl: 0.06550323370469974
equity: -0.05810467428222364, infl: 0.04050950201588278
equity: 0.17308855652609606, infl: 0.04004276358559344


In [100]:
clusters = assignments(result)

99999-element Vector{Int64}:
 12
  6
 11
  6
 11
 12
  8
  1
  9
  6
  ⋮
  9
  9
 11
  4
  2
  2
 10
  2
  7

In [101]:
cluster_means = zeros((2,n_clusters))

for cluster in 1:n_clusters
    cluster_means[:,cluster] = mean(period_1[:,clusters .== cluster],dims=2) 
end


for cluster in 1:n_clusters
    println("$(cluster_means[1,cluster]), $(cluster_means[2,cluster])")
end

0.22529584840447545, 0.040862305014503796
-0.1410049027071038, 0.041045115204559654
0.46778991143654186, 0.04044485634588581
0.35790638775510103, 0.03986975836734742
-0.25537023039215734, 0.0403465911764707
0.06146688934217203, 0.03808216012238544
0.006285096140412094, 0.04090456845965686
0.119784838425926, 0.024867299907408352
0.2843733376578649, 0.040957500344431436
0.11451576498463137, 0.06553850755635249
-0.058285417094644236, 0.04051099009537791
0.1730425081591743, 0.039970957557971865


In [102]:
probs = counts(result)/sum(counts(result))

12-element Vector{Float64}:
 0.12065120651206512
 0.04913049130491305
 0.014340143401434014
 0.049000490004900046
 0.02040020400204002
 0.13727137271372714
 0.11452114521145211
 0.10800108001080011
 0.08710087100871008
 0.07808078080780807
 0.08178081780817809
 0.13972139721397214

In [103]:
scenario_period1 = DataFrame(Equity=cluster_means[1,:].*100,EDO=cluster_means[2,:].*100,Probs=probs*100)

Row,Equity,EDO,Probs
,Float64,Float64,Float64
1,22.5296,4.08623,12.0651
2,-14.1005,4.10451,4.91305
3,46.779,4.04449,1.43401
4,35.7906,3.98698,4.90005
5,-25.537,4.03466,2.04002
6,6.14669,3.80822,13.7271
7,0.62851,4.09046,11.4521
8,11.9785,2.48673,10.8001
9,28.4373,4.09575,8.71009


In [216]:
period_1

2×99999 Matrix{Float64}:
 0.183608  0.057796  -0.042034  0.048233  …  0.119115  -0.145221  -0.025328
 0.035158  0.013222   0.039912  0.016048     0.064728   0.027103   0.0123

In [270]:
function cluster(data, n_clusters)
    dt = fit(ZScoreTransform, data, dims=2)
    standarized = StatsBase.transform(dt,data)

    clusters = kmeans(standarized,n_clusters)

    destandarised = StatsBase.reconstruct(dt, clusters.centers)

    probs = counts(clusters)/sum(counts(clusters))

    return destandarised, probs
end

grid, probs = cluster(period_1, 6)
display(DataFrame(grid, :auto))
display(probs)

Row,x1,x2,x3,x4,x5,x6
,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.00586575,0.100723,-0.100866,0.238938,0.287256,0.109974
2,0.0708532,0.0180867,0.0301647,0.06814,0.0271872,0.0451161


6-element Vector{Float64}:
 0.11873118731187311
 0.21935219352193522
 0.12762127621276212
 0.13658136581365812
 0.16708167081670816
 0.23063230632306322

In [107]:
source_data = TimeArray(CSV.File("assets_data_03.24.csv", delim=';', dateformat=Dict("Date" => "dd.mm.yyyy"), decimal=','),timestamp=:Date)
returns = percentchange(source_data, :log)


615×5 TimeArray{Union{Missing, Float64}, 2, Date, Matrix{Union{Missing, Float64}}} 1973-01-31 to 2024-03-31
┌────────────┬────────────┬─────────────┬──────────────┬──────────────┬─────────
│            │ ACWI_Net   │ USDPLN      │ ACWI_Net_PLN │ CPI_USA      │ CPI_PL ⋯
├────────────┼────────────┼─────────────┼──────────────┼──────────────┼─────────
│ 1973-01-31 │    missing │     missing │      missing │   0.00235067 │     mi ⋯
│ 1973-02-28 │    missing │     missing │      missing │   0.00701905 │     mi ⋯
│ 1973-03-31 │    missing │     missing │      missing │   0.00927727 │     mi ⋯
│ 1973-04-30 │    missing │     missing │      missing │   0.00690598 │     mi ⋯
│ 1973-05-31 │    missing │     missing │      missing │   0.00685862 │     mi ⋯
│ 1973-06-30 │    missing │     missing │      missing │    0.0068119 │     mi ⋯
│ 1973-07-31 │    missing │     missing │      missing │   0.00225501 │     mi ⋯
│ 1973-08-31 │    missing │     missing │      missing │    0.0179014 │     mi ⋯
│

In [108]:
returns_yearly = percentchange(collapse(source_data,year, last))

52×5 TimeArray{Union{Missing, Float64}, 2, Date, Matrix{Union{Missing, Float64}}} 1973-12-31 to 2024-03-31
┌────────────┬────────────┬────────────┬──────────────┬────────────┬────────────
│            │ ACWI_Net   │ USDPLN     │ ACWI_Net_PLN │ CPI_USA    │ CPI_PLN   ⋯
├────────────┼────────────┼────────────┼──────────────┼────────────┼────────────
│ 1973-12-31 │    missing │    missing │      missing │  0.0870605 │     missi ⋯
│ 1974-12-31 │    missing │    missing │      missing │   0.123377 │     missi ⋯
│ 1975-12-31 │    missing │    missing │      missing │  0.0693605 │     missi ⋯
│ 1976-12-31 │    missing │    missing │      missing │  0.0486505 │     missi ⋯
│ 1977-12-31 │    missing │    missing │      missing │  0.0670123 │     missi ⋯
│ 1978-12-31 │    missing │    missing │      missing │   0.090177 │     missi ⋯
│ 1979-12-31 │    missing │    missing │      missing │   0.132939 │     missi ⋯
│ 1980-12-31 │    missing │    missing │      missing │   0.125161 │     missi ⋯
│ 

In [271]:
val = eachcol(values(from(returns_yearly,Date(2003,02,01))))

[percentile(a,[5, 25, 50, 75, 95]) for a in val]

5-element Vector{Vector{Float64}}:
 [-0.1791646535637412, 0.050866825980199704, 0.13951720841975523, 0.21889844750843207, 0.3361762844747671]
 [-0.16789023904324368, -0.08309423208140795, 0.01405036560292702, 0.08138626868521559, 0.17291389833351883]
 [-0.11150993397204947, 0.03390535467137583, 0.09733228505343647, 0.19875932356089762, 0.30291930341482903]
 [0.007308352896474506, 0.015616012312351955, 0.02091860212971578, 0.03182293580839392, 0.06335757465785029]
 [-0.005813596241253867, 0.00972740828317448, 0.02302606076369674, 0.03842710017413295, 0.08567916010546725]

In [121]:
acwi_cpipln = identity.(values(from(returns[:ACWI_Net_PLN,:CPI_PLN],Date(2003,02,01))))

254×2 Matrix{Float64}:
  0.0051036     0.000927653
  0.0430686     0.00277912
  0.0118412     0.0018489
  0.0300994     0.0
  0.0670022    -0.000924682
  0.0132941    -0.00370334
  0.0495565    -0.00371711
  0.000195397   0.0046439
  0.0782452     0.00554541
 -0.0161735     0.00276122
  ⋮            
  0.0208096    -0.00221661
  0.000986609   0.0
  0.0154043    -0.00389282
 -0.066645      0.00278164
  0.0356998     0.00719694
  0.0310713     0.00110186
  0.0233511     0.00384933
  0.0394925     0.00274048
  0.0298368     0.00218699

In [218]:
transpose(acwi_cpipln)

2×254 transpose(::Matrix{Float64}) with eltype Float64:
 0.0051036    0.0430686   0.0118412  …  0.0233511   0.0394925   0.0298368
 0.000927653  0.00277912  0.0018489     0.00384933  0.00274048  0.00218699

In [229]:
state, probs = cluster(transpose(acwi_cpipln),4)
display(state*12)
display(probs)


2×4 Matrix{Float64}:
 -0.021201   -0.0412958  -0.741961   0.506915
  0.0061562   0.138942    0.0357811  0.0235401

4-element Vector{Float64}:
 0.4409448818897638
 0.12992125984251968
 0.09055118110236221
 0.33858267716535434

In [268]:
acwi_cpiusd = identity.(values(from(returns[:ACWI_Net,:CPI_USA],Date(1988,01,01))))

state, probs = cluster(transpose(acwi_cpiusd),3)

display(state)
display(probs)

2×3 Matrix{Float64}:
 0.0167981   -0.0502935    0.0307706
 0.00519076   0.00202026  -0.000120185

3-element Vector{Float64}:
 0.3563218390804598
 0.2413793103448276
 0.40229885057471265